In [1]:
########
# LC generator for RRLyrs
########

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bg_color = "#000000"
use_period_color = True
R_color = 1
G_color = 1
B_color = 1
var_Type = "rrlyr"
pltSIZE = 128
pltDPI = 72
stackSize = 25000
generateClones = False

webaddress_base = "http://www.astrouw.edu.pl/ogle/ogle4/OCVS/lmc/" + var_Type + "/"
web_data_folder = "phot/I/OGLE-LMC-" + var_Type.upper() + "-"

ident = pd.read_csv(webaddress_base + "ident.dat", header=None, delim_whitespace=True, usecols=[0,3])
ident.columns = ['id','type']

datacount = ident.shape[0]
cloneGenRatio = int((stackSize-datacount)/datacount)
cloneNumber = stackSize-datacount

rrab_data = pd.read_csv(webaddress_base + "RRab.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
rrc_data = pd.read_csv(webaddress_base + "RRc.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
# rre_data = pd.read_csv(webaddress_base + "RRe.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
rrd_data = pd.read_csv(webaddress_base + "RRd.dat", header=None, delim_whitespace=True, usecols=[0,11,13])
rrd_data.columns = [0,3,5]

# types = [rrab_data,rrc_data,rre_data,rrd_data]
types = [rrab_data,rrc_data,rrd_data]

rrlyr_data = pd.concat(types,ignore_index=True)

rrlyr_data.reset_index(drop=True)
rrlyr_data.columns = ['id','period','T_max']
rrlyr_data['id'] = pd.Series(rrlyr_data['id']).str[15:20]
rrlyr_data.set_index('id', inplace=True)

#rrlyr_data['id'] = pd.to_numeric(rrlyr_data['id'])
#rrlyr_data = rrlyr_data.to_numpy(copy=True)

In [3]:
print("Available data:", datacount)
print("Desired number of data:", stackSize)
print("Will generate", cloneNumber , "clones")

Available data: 41116
Desired number of data: 25000
Will generate -16116 clones


In [6]:
webaddress = webaddress_base + web_data_folder

if generateClones:
    cloneCounter = 0

parentCounter = 0

for rrlrow in range(31102,32510):
#    if ident['type'][rrlrow] != "RRd":
#        continue
#    else:
    star_identifier = str(f'{rrlrow+1:05d}')
    data_filename = webaddress + star_identifier + ".dat"

    try:
        data = pd.read_csv(data_filename, delim_whitespace=True, header=None, usecols=[0,1,2])
    except:
        print("No variable found at " + data_filename)
        continue
    data.columns = ["date", "mag","error"]
                
    T_max = np.float(rrlyr_data.loc[star_identifier].T_max)
    period = np.float(rrlyr_data.loc[star_identifier].period)
    print(T_max, "--", period)

    if T_max > data['date'][0]:
        t_null = data['date'][0]
        pr = np.ceil((T_max - t_null)/period)
#         print(pr)
        T_max = T_max - pr*period
#         print(T_max)

    if use_period_color:
#         print("Color coded period")
        if 0 < period and period <=1:
            R_color = period
            G_color = 0
            B_color = 0
        if 1 < period and period <=10:
            R_color = 1
            G_color = period/10
            B_color = 0
        if 10 < period and period <=100:
            R_color = 1
            G_color = 1
            B_color = period/100
        if period > 100:
            R_color = 1
            G_color = 1
            B_color = 1
    else:
#         print("Not using color coded period")
        print("Color: R:", R_color, "G:", G_color, "B:", B_color)

    
#     print("Color: R:", R_color, "G:", G_color, "B:", B_color)
#     print(T_max, "--", period)
        
    flux_cont = []
    flux_norm_cont = []
    phase_cont = []
    
    mag_cont = []
    t_cont = []
    error_cont = []
    
    for i in data.index:
        mag = data['mag'][i]
        t = data['date'][i]
        error = data['error'][i]
        mag_cont.append(mag)
        t_cont.append(t)
        error_cont.append(error)
        
        phase = (t-T_max)/period - int((t-T_max)/period)
        flux = 10**(-0.4*(mag-25))
        flux_cont.append(flux)
        phase_cont.append(phase)
        #print(mag , "--",  flux)
        
    if generateClones and cloneCounter != cloneNumber:
#         if row == 0:
#             cloneGenRatio = round((stackSize-datacount)/datacount)
#             floatRatio = (stackSize-datacount)/datacount
#         else:
#             # Checking and adjusting clone number
#             remParent = datacount-parentCounter
#             remAll = (stackSize-cloneCounter-parentCounter)-(datacount-parentCounter)
#             cloneGenRatio = round(remAll/remParent)
#             floatRatio = remAll/remParent
        cloneGenRatio = 1
        
        print("===================")
        max_error = max(error_cont)
        print("Max error:", max_error)
        error_cont = np.random.normal(0, max_error/3, len(error_cont))
        print("Error array generated for " + var_Type + "_" + star_identifier)
         
        for cl in range(cloneGenRatio):
            if cloneCounter == cloneNumber:
                continue
                
            flux_clone_cont = []
            flux_clone_norm_cont = []
            rand_error_array = np.random.choice(error_cont,len(error_cont))
            mag_clone_cont = mag_cont + rand_error_array
            
            print("Clone generation", cl)
            for c in range(len(mag_clone_cont)):
                flux_clone = 10**(-0.4*(mag_clone_cont[c]-25))
                flux_clone_cont.append(flux_clone)
            
            max_clone_flux = max(flux_clone_cont)
            for i in flux_clone_cont:
                flux_clone_norm = (i/max_clone_flux)
                flux_clone_norm_cont.append(flux_clone_norm)

            phase_cont = np.array(phase_cont)
            flux_clone_norm_cont = np.array(flux_clone_norm_cont)
            
            ###     
            # Plotting the folded lightcurve for cloned variable
            ### 
            plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

            plt.plot(phase_cont, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
            plt.plot(phase_cont+1, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

            plt.axis('off')

        #     period_str = str(period).replace(".","_")
        #     period_str = period_str[0:6]
        #     filename = "ACEP_LCs/" + var_Type + "_" + star_identifier + ".png"
            filename = "../testdata_nocolor/" + var_Type + "_" + star_identifier + "_clone_" + str(cl) +".png"

        #     filename = "ACEP_LCs/" + period_str + "_" + var_Type + "_" + star_identifier + ".png"
        #     filename = "ACEP_LCs/" + period_str + ".png"
            print("Saving " + filename)
            plt.tight_layout(0)
            plt.gcf().savefig(filename, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#             plt.show()
            plt.close()
            
            cloneCounter += 1
        
            ###     
            # End of plotting for the clone(s)
            ### 

# 
# Creating data for the original variable star
# 
    max_flux = max(flux_cont)

    for i in flux_cont:
        flux_norm = (i/max_flux)
        flux_norm_cont.append(flux_norm)

#if len(phase_cont) == len(flux_norm_cont):
    phase_cont = np.array(phase_cont)
    flux_norm_cont = np.array(flux_norm_cont)

    plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

    plt.plot(phase_cont, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
    plt.plot(phase_cont+1, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

    plt.axis('off')

#     period_str = str(period).replace(".","_")
#     period_str = period_str[0:6]
    filename = "../testdata_color/" + var_Type + "_" + star_identifier + ".png"
#     filename = "../colored_lcs/" + var_Type + "_" + star_identifier + ".png"

#     filename = "RRLyr_LCs/" + period_str + "_" + var_Type + "_" + star_identifier + ".png"
#     filename = "RRLyr_LCs/" + period_str + ".png"
    print("Saving ", filename)
    plt.tight_layout(0)
    plt.gcf().savefig(filename, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#       plt.show()
    plt.close()


6000.34982 -- 0.3609352
Saving  ../testdata_color/rrlyr_31103.png
6000.224389999999 -- 0.5251515999999999
Saving  ../testdata_color/rrlyr_31104.png
6000.02005 -- 0.7398057
Saving  ../testdata_color/rrlyr_31105.png
6000.12464 -- 0.5338566
Saving  ../testdata_color/rrlyr_31106.png
6000.20356 -- 0.4996435
Saving  ../testdata_color/rrlyr_31107.png
6000.09618 -- 0.6165888
Saving  ../testdata_color/rrlyr_31108.png
6000.07477 -- 0.7645736
Saving  ../testdata_color/rrlyr_31109.png
6000.41096 -- 0.5847002
Saving  ../testdata_color/rrlyr_31110.png
6000.37635 -- 0.4893419
Saving  ../testdata_color/rrlyr_31111.png
6000.481720000001 -- 0.6304764
Saving  ../testdata_color/rrlyr_31112.png
6000.1282200000005 -- 0.5854742
Saving  ../testdata_color/rrlyr_31113.png
6000.23106 -- 0.5891900999999999
Saving  ../testdata_color/rrlyr_31114.png
6000.3708 -- 0.47225
Saving  ../testdata_color/rrlyr_31115.png
6000.31301 -- 0.5956935
Saving  ../testdata_color/rrlyr_31116.png
6000.40675 -- 0.5054922000000001
Saving

6000.33247 -- 0.6457446
Saving  ../testdata_color/rrlyr_31221.png
6000.232779999999 -- 0.602592
Saving  ../testdata_color/rrlyr_31222.png
6000.16634 -- 0.4806657999999999
Saving  ../testdata_color/rrlyr_31223.png
6000.16558 -- 0.2806686
Saving  ../testdata_color/rrlyr_31224.png
6000.12814 -- 0.5660482
Saving  ../testdata_color/rrlyr_31225.png
6000.3316 -- 0.6565473000000001
Saving  ../testdata_color/rrlyr_31226.png
6000.04894 -- 0.37067540000000004
Saving  ../testdata_color/rrlyr_31227.png
6000.26102 -- 0.5362143
Saving  ../testdata_color/rrlyr_31228.png
6000.1509399999995 -- 0.7103569000000001
Saving  ../testdata_color/rrlyr_31229.png
6000.03551 -- 0.6454878000000001
Saving  ../testdata_color/rrlyr_31230.png
6000.0449 -- 0.545933
Saving  ../testdata_color/rrlyr_31231.png
6000.00549 -- 0.5052578
Saving  ../testdata_color/rrlyr_31232.png
6000.14749 -- 0.5811598000000001
Saving  ../testdata_color/rrlyr_31233.png
6000.00295 -- 0.5361326999999999
Saving  ../testdata_color/rrlyr_31234.png
6

Saving  ../testdata_color/rrlyr_31338.png
6000.17735 -- 0.5199252999999999
Saving  ../testdata_color/rrlyr_31339.png
6000.1822 -- 0.3099143
Saving  ../testdata_color/rrlyr_31340.png
6000.01775 -- 0.5028607
Saving  ../testdata_color/rrlyr_31341.png
6000.27838 -- 0.5791947
Saving  ../testdata_color/rrlyr_31342.png
6000.08891 -- 0.5764864000000001
Saving  ../testdata_color/rrlyr_31343.png
6000.26278 -- 0.633698
Saving  ../testdata_color/rrlyr_31344.png
6000.10491 -- 0.2848491
Saving  ../testdata_color/rrlyr_31345.png
6000.03307 -- 0.2749386
Saving  ../testdata_color/rrlyr_31346.png
6000.27296 -- 0.5994009
Saving  ../testdata_color/rrlyr_31347.png
6000.09106 -- 0.6237224
Saving  ../testdata_color/rrlyr_31348.png
6000.05482 -- 0.6185522
Saving  ../testdata_color/rrlyr_31349.png
6000.061 -- 0.37517249999999996
Saving  ../testdata_color/rrlyr_31350.png
6000.50858 -- 0.6721751
Saving  ../testdata_color/rrlyr_31351.png
6000.14927 -- 0.6214674
Saving  ../testdata_color/rrlyr_31352.png
6000.03932

Saving  ../testdata_color/rrlyr_31455.png
6000.48313 -- 0.48937709999999995
Saving  ../testdata_color/rrlyr_31456.png
6000.50942 -- 0.5434234
Saving  ../testdata_color/rrlyr_31457.png
6000.33615 -- 0.598928
Saving  ../testdata_color/rrlyr_31458.png
6000.00793 -- 0.2963872
Saving  ../testdata_color/rrlyr_31459.png
6000.07152 -- 0.5416536
Saving  ../testdata_color/rrlyr_31460.png
6000.34914 -- 0.3572035
Saving  ../testdata_color/rrlyr_31461.png
6000.021320000001 -- 0.5614914000000001
Saving  ../testdata_color/rrlyr_31462.png
6000.001929999999 -- 0.7412297
Saving  ../testdata_color/rrlyr_31463.png
6000.11955 -- 0.33543579999999995
Saving  ../testdata_color/rrlyr_31464.png
6000.31796 -- 0.6675601999999999
Saving  ../testdata_color/rrlyr_31465.png
6000.43729 -- 0.4924001
Saving  ../testdata_color/rrlyr_31466.png
6000.1717100000005 -- 0.304507
Saving  ../testdata_color/rrlyr_31467.png
6000.36042 -- 0.5573811
Saving  ../testdata_color/rrlyr_31468.png
6000.5596399999995 -- 0.6046832
Saving  ..

6000.30535 -- 0.45647309999999996
Saving  ../testdata_color/rrlyr_31573.png
6000.328729999999 -- 0.6400704
Saving  ../testdata_color/rrlyr_31574.png
6000.13602 -- 0.3581267
Saving  ../testdata_color/rrlyr_31575.png
6000.12869 -- 0.3501488
Saving  ../testdata_color/rrlyr_31576.png
6000.47234 -- 0.502554
Saving  ../testdata_color/rrlyr_31577.png
6000.38098 -- 0.570181
Saving  ../testdata_color/rrlyr_31578.png
6000.2381 -- 0.5528933000000001
Saving  ../testdata_color/rrlyr_31579.png
6000.23308 -- 0.6660775
Saving  ../testdata_color/rrlyr_31580.png
6000.2037 -- 0.4748767
Saving  ../testdata_color/rrlyr_31581.png
6000.064 -- 0.5962263000000001
Saving  ../testdata_color/rrlyr_31582.png
6000.15448 -- 0.5125294
Saving  ../testdata_color/rrlyr_31583.png
6000.37613 -- 0.5757465
Saving  ../testdata_color/rrlyr_31584.png
6000.3285399999995 -- 0.3411651
Saving  ../testdata_color/rrlyr_31585.png
6000.29529 -- 0.6796449000000001
Saving  ../testdata_color/rrlyr_31586.png
6000.472229999999 -- 0.6161369

...

6000.1449 -- 0.36897470000000004
Saving  ../testdata_color/rrlyr_32040.png
6000.4643399999995 -- 0.48359589999999997
Saving  ../testdata_color/rrlyr_32041.png
6000.18317 -- 0.573892
Saving  ../testdata_color/rrlyr_32042.png
6000.432589999999 -- 0.5885204
Saving  ../testdata_color/rrlyr_32043.png
6000.162060000001 -- 0.5104849
Saving  ../testdata_color/rrlyr_32044.png
6000.02953 -- 0.28146509999999997
Saving  ../testdata_color/rrlyr_32045.png
6000.1939600000005 -- 0.6131787
Saving  ../testdata_color/rrlyr_32046.png
6000.43937 -- 0.5114388
Saving  ../testdata_color/rrlyr_32047.png
6000.28558 -- 0.3391012
Saving  ../testdata_color/rrlyr_32048.png
6000.32429 -- 0.3568881
Saving  ../testdata_color/rrlyr_32049.png
6000.04753 -- 0.65062
Saving  ../testdata_color/rrlyr_32050.png
6000.038979999999 -- 0.5340354
Saving  ../testdata_color/rrlyr_32051.png
6000.26574 -- 0.5250104
Saving  ../testdata_color/rrlyr_32052.png
6000.14385 -- 0.3685291
Saving  ../testdata_color/rrlyr_32053.png
6000.09275 --

6000.12024 -- 0.35626620000000003
Saving  ../testdata_color/rrlyr_32157.png
6000.57568 -- 0.641925
Saving  ../testdata_color/rrlyr_32158.png
6000.34937 -- 0.5172554
Saving  ../testdata_color/rrlyr_32159.png
6000.3424700000005 -- 0.534287
Saving  ../testdata_color/rrlyr_32160.png
6000.2397 -- 0.6272496
Saving  ../testdata_color/rrlyr_32161.png
6000.01238 -- 0.2991188
Saving  ../testdata_color/rrlyr_32162.png
6000.320720000001 -- 0.5846265
Saving  ../testdata_color/rrlyr_32163.png
6000.11247 -- 0.5028710999999999
Saving  ../testdata_color/rrlyr_32164.png
6000.595789999999 -- 0.7072554
Saving  ../testdata_color/rrlyr_32165.png
6000.13555 -- 0.28149009999999997
Saving  ../testdata_color/rrlyr_32166.png
6000.34103 -- 0.5748239
Saving  ../testdata_color/rrlyr_32167.png
6000.19468 -- 0.7646386
Saving  ../testdata_color/rrlyr_32168.png
6000.40201 -- 0.5832515
Saving  ../testdata_color/rrlyr_32169.png
6000.33172 -- 0.5010789999999999
Saving  ../testdata_color/rrlyr_32170.png
6000.539070000001 -

Saving  ../testdata_color/rrlyr_32273.png
6000.31568 -- 0.5209817
Saving  ../testdata_color/rrlyr_32274.png
6000.26953 -- 0.5224245
Saving  ../testdata_color/rrlyr_32275.png
6000.13479 -- 0.565876
Saving  ../testdata_color/rrlyr_32276.png
6000.29151 -- 0.7678429
Saving  ../testdata_color/rrlyr_32277.png
6000.01065 -- 0.5664201
Saving  ../testdata_color/rrlyr_32278.png
6000.29381 -- 0.39296709999999996
Saving  ../testdata_color/rrlyr_32279.png
6000.10883 -- 0.5808386
Saving  ../testdata_color/rrlyr_32280.png
6000.05201 -- 0.367298
Saving  ../testdata_color/rrlyr_32281.png
6000.21603 -- 0.5878998000000001
Saving  ../testdata_color/rrlyr_32282.png
6000.4133 -- 0.442043
Saving  ../testdata_color/rrlyr_32283.png
6000.0941 -- 0.6510359
Saving  ../testdata_color/rrlyr_32284.png
6000.02014 -- 0.5449316
Saving  ../testdata_color/rrlyr_32285.png
6000.22512 -- 0.6943954999999999
Saving  ../testdata_color/rrlyr_32286.png
6000.09858 -- 0.617101
Saving  ../testdata_color/rrlyr_32287.png
6000.3043700

Saving  ../testdata_color/rrlyr_32392.png
6000.4938 -- 0.5906474
Saving  ../testdata_color/rrlyr_32393.png
6000.12645 -- 0.5331661
Saving  ../testdata_color/rrlyr_32394.png
6000.21583 -- 0.23566610000000002
Saving  ../testdata_color/rrlyr_32395.png
6000.13811 -- 0.5027786
Saving  ../testdata_color/rrlyr_32396.png
6000.01455 -- 0.5879236999999999
Saving  ../testdata_color/rrlyr_32397.png
6000.37053 -- 0.5511843000000001
Saving  ../testdata_color/rrlyr_32398.png
6000.28505 -- 0.3433992
Saving  ../testdata_color/rrlyr_32399.png
6000.19757 -- 0.30276490000000006
Saving  ../testdata_color/rrlyr_32400.png
6000.25227 -- 0.5379928
Saving  ../testdata_color/rrlyr_32401.png
6000.68318 -- 0.7130032
Saving  ../testdata_color/rrlyr_32402.png
6000.32837 -- 0.385477
Saving  ../testdata_color/rrlyr_32403.png
6000.11906 -- 0.5856022
Saving  ../testdata_color/rrlyr_32404.png
6000.2919 -- 0.4742988
Saving  ../testdata_color/rrlyr_32405.png
6000.00213 -- 0.2712299
Saving  ../testdata_color/rrlyr_32406.png

Saving  ../testdata_color/rrlyr_32510.png
